In [122]:
import pandas as pd

### Load data sources

- list of schools
- locality
- counties
- Bac Data

In [218]:
# Bac Data 
data_bac = pd.read_csv("../data/CleanData/data_bac.csv")

# list of school
school_norm_df = pd.read_csv("../data/CleanData/list_schools_upd.csv")
school_norm_df = school_norm_df.rename(columns={"location_ID": "locality_id", "unit_SIIIR": "school_id"})

# locality
regions_df = pd.read_csv("../data/CleanData/list_regions.csv")
regions_df = regions_df.rename(columns={"location_ID": "locality_id", "county": "county_name"})

C:\Users\Alexandra\AppData\Local\Temp\ipykernel_16704\190747967.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_bac = pd.read_csv("../data/CleanData/data_bac.csv")


In [ ]:
# ************ geography ***********
# Split region table in locality and county
county_df = regions_df[["county_name"]].dropna().drop_duplicates()
county_df["county_id"] = range(1, len(county_df) + 1)

locality_df = regions_df[["locality", "county_name", "locality_id"]].drop_duplicates()
locality_df = locality_df.merge(county_df, on="county_name", how="left")
locality_df = locality_df.rename(columns={"locality": "locality_name"})
locality_df = locality_df[["locality_id", "locality_name", "county_id"]]


In [219]:
data_bac = data_bac.rename(columns= {
    'candidate_unique_code': 'student_code',
    'subject_ea': 'core_subject_1',
    'subject_ec': 'core_subject_2',
    'subject_ed': 'elective_subject_name',
    'status_ea': 'status_subject_1',
    'status_ec': 'status_subject_2',
    'status_ed': 'status_elective_subject',
    'final_score_ea': 'final_score_subject_1',
    'final_score_ec': 'final_score_subject_2',
    'final_score_ed': 'final_score_elective_subject',
    'profile': 'profile_name',
    'form_of_education': 'form_of_education_name',
    'field_of_study': 'field_of_study_name',
    'modern_language': 'language_name',
    'unit_SIIIR': 'school_id',
    }
)

In [158]:
# ************ gender/environment/form of education/field of study ***********

gender_df = pd.DataFrame(data_bac["gender"].dropna().unique(), columns=["gender_type"])
gender_df["gender_id"] = range(1, len(gender_df) + 1)

environment_df = pd.DataFrame(data_bac["candidate_environment"].dropna().unique(), columns=["environment_type"])
environment_df["environment_id"] = range(1, len(environment_df) + 1)

form_of_education_df = pd.DataFrame(data_bac["form_of_education_name"].dropna().unique(), columns=["form_of_education_name"])
form_of_education_df["form_of_education_id"] = range(1, len(form_of_education_df) + 1)

field_of_study_df = pd.DataFrame(data_bac["field_of_study_name"].dropna().unique(), columns=["field_of_study_name"])
field_of_study_df["field_of_study_id"] = range(1, len(field_of_study_df) + 1)

In [159]:
# ************ modern language/ elective subject / profile ***********
modern_language_df = pd.DataFrame(data_bac["language_name"].dropna().unique(), columns=["language_name"])
modern_language_df["language_id"] = range(1, len(modern_language_df) + 1)

profile_df = pd.DataFrame(data_bac["profile_name"].dropna().unique(), columns=["profile_name"])
profile_df["profile_id"] = range(1, len(profile_df) + 1)

elective_subject_df = data_bac[["elective_subject_name"]].dropna().drop_duplicates()
elective_subject_df["elective_subject_id"] = range(1, len(elective_subject_df) + 1)

In [226]:
# ************ specialization ************ 
data_bac["specialization_id"] = (
    data_bac
    .groupby(
        ["specialization", "core_subject_1", "core_subject_2", "elective_subject_name", "profile_name"],
        dropna=False
    )
    .ngroup() + 1
)

# specialization lookup table
specialization_df = data_bac[
    ["specialization_id", "specialization", "core_subject_1", "core_subject_2", "elective_subject_name", "profile_name"]
].drop_duplicates()

specialization_df = specialization_df.merge(
    elective_subject_df, on="elective_subject_name", how="left"
).merge(
    profile_df, on="profile_name", how="left"
)

specialization_df = specialization_df.rename(columns={"specialization": "specialization_name"})
specialization_df = specialization_df[
    ["specialization_id", "specialization_name", "core_subject_1", "core_subject_2", "elective_subject_id", "profile_id"]
]


In [224]:
# ************ grades ************ 
grades_df = data_bac[
    [
        "student_code","status", "average", "year", "session", "digital_score",
        "status_elective_subject", "final_score_elective_subject",
        "status_subject_1", "final_score_subject_1",
        "status_subject_2", "final_score_subject_2",
        "status_c", 'ita', 'written_itc', 'written_pms', 'oral_pmo', 'oral_io',
        "status_d", "status_a", "status_b"

    ]
].drop_duplicates()


grades_df = grades_df.rename(columns= {"status_d": "status_digital", "status_c": "modern_language_status_c", "ita": "modern_language_listening", "written_itc": "modern_language_written_reading_comprehension",
         "written_pms": "modern_language_written_production", "oral_pmo": "modern_language_oral_production", "oral_io": "modern_language_oral_oi", "status_a": "status_ro_oral", "status_b": "status_ro_written"})


grades_df["grades_id"] = range(1, len(grades_df) + 1)


In [231]:
# ---------- student ----------

student_df = data_bac.copy()

student_df["student_id"] = (
    student_df
    .groupby(
        ["student_code", "session"],
        dropna=False
    )
    .ngroup() + 1
)


student_df = student_df.merge(gender_df, left_on="gender", right_on="gender_type", how="left") \
    .merge(environment_df, left_on="candidate_environment", right_on="environment_type", how="left") \
    .merge(form_of_education_df, on="form_of_education_name", how="left") \
    .merge(field_of_study_df, on="field_of_study_name", how="left") \
    .merge(modern_language_df, on="language_name", how="left") \
    .merge(school_norm_df, on="school_id", how="left") \
    .merge(grades_df, on=["student_code"], how="left")


student_df = student_df[
    [
        "student_id","student_code", "graduation_year", "form_of_education_id", "school_id",
        "gender_id", "environment_id", "class_year", "field_of_study_id",
        "language_id", "specialization_id", "grades_id"
    ]
]

student_df = student_df.drop_duplicates(subset=["student_id"])


### Export data to SQL

In [137]:
from sqlalchemy import create_engine

In [ ]:
engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [142]:
field_of_study_df.to_sql('field_of_study', con=engine, if_exists='append', index=False)

3

In [144]:
form_of_education_df.to_sql('form_of_education', con=engine, if_exists='append', index=False)

3

In [145]:
environment_df.to_sql('environment', con=engine, if_exists='append', index=False)

2

In [163]:
county_df.to_sql('county', con=engine, if_exists='append', index=False)

42

In [164]:
profile_df.to_sql('profile', con=engine, if_exists='append', index=False)

10

In [165]:
elective_subject_df.to_sql('elective_subject', con=engine, if_exists='append', index=False)

20

In [ ]:
grades_df = grades_df.drop(columns=['student_code'])
grades_df.to_sql('grades', con=engine, if_exists='append', index=False)

1264056

In [179]:
modern_language_df.to_sql('modern_language', con=engine, if_exists='append', index=False)

11

In [180]:
gender_df.to_sql('gender', con=engine, if_exists='append', index=False)

2

In [ ]:
locality_df = locality_df[["locality_id", "locality_name", "county_id"]]
locality_df.to_sql('locality', con=engine, if_exists='append', index=False)

7170

In [195]:
school_norm_df.to_sql('school', con=engine, if_exists='append', index=False)

1381

In [199]:
specialization_df.to_sql('specialization', con=engine, if_exists='append', index=False)

527

In [235]:
student_df.to_sql('student', con=engine, if_exists='append', index=False)

1149939